In [ ]:
import shutil
import os

def copy_directories(src_root, dest_root):
    # Create destination root directory if it doesn't exist
    os.makedirs(dest_root, exist_ok=True)

    # Walk through the source directory
    for dirpath, dirnames, filenames in os.walk(src_root):
        # Create corresponding directory structure in the destination
        relative_path = os.path.relpath(dirpath, src_root)
        dest_dir = os.path.join(dest_root, relative_path)
        os.makedirs(dest_dir, exist_ok=True)

        # Copy files to the destination directory
        for filename in filenames:
            src_file = os.path.join(dirpath, filename)
            dest_file = os.path.join(dest_dir, filename)
            shutil.copy2(src_file, dest_file)

# Define source and destination directories
source_directory = '/kaggle/input/cric-pose-v2/cric_pose_v2'
destination_directory = '/kaggle/working/cricshot-yolo-ano'

# Copy the directories and files
copy_directories(source_directory, destination_directory)


In [ ]:
!pip install -q ultralytics
!pip install -q ffmpeg-python


In [ ]:
import ffmpeg
import os
import shutil
import cv2
from ultralytics import YOLO

def convert_to_yolo_annotated_vid(model, src, des):
    # Directory to save the processed frames
    output_dir = '/kaggle/working/processed_frames'
    os.makedirs(output_dir, exist_ok=True)

    # Open the video file
    cap = cv2.VideoCapture(src)

    # Initialize frame counter
    frame_number = 0

    # Process video frames
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Zoom factor (e.g., 1.2 means a 20% zoom)
        zoom_factor = 1.0

        # Get the frame dimensions
        h, w, _ = frame.shape

        # Calculate the cropping area
        crop_h = int(h / zoom_factor)
        crop_w = int(w / zoom_factor)
        start_h = (h - crop_h) // 2
        start_w = (w - crop_w) // 2

        # Crop and resize the frame
        zoomed_frame = frame[start_h:start_h + crop_h, start_w:start_w + crop_w]
        zoomed_frame = cv2.resize(zoomed_frame, (w, h))

        # Run YOLOv8 inference on the zoomed frame
        results = model(zoomed_frame)
        
        # Extract keypoints and ensure no bounding boxes are displayed
        for result in results:
            # Remove bounding boxes by setting them to None or empty list
            result.boxes = None  # Disables bounding box rendering

        # Visualize the results on the zoomed frame
        annotated_frame = results[0].plot()

        # Save the annotated frame as an image
        output_frame_path = os.path.join(output_dir, f"frame_{frame_number:04d}.jpg")
        cv2.imwrite(output_frame_path, annotated_frame)
        frame_number += 1

    # Release the video capture object
    cap.release()

    # Define the path to the input frames and the output video file
    input_frames_path = os.path.join(output_dir, 'frame_%04d.jpg')
    output_video_path = des

    # Set the frame rate
    frame_rate = 32

    # Use ffmpeg to create the video
    ffmpeg.input(input_frames_path, framerate=frame_rate)\
          .output(output_video_path, vcodec='libx264', pix_fmt='yuv420p')\
          .overwrite_output()\
          .run()

    # Remove all files in the processed_frames directory
    shutil.rmtree(output_dir)
    
    print(f"Video saved at {output_video_path}")

def process_videos_in_directory(model, root_dir):
    # Walk through the directory structure
    for dirpath, dirnames, filenames in os.walk(root_dir):
        for filename in filenames:
            if filename.endswith('.mp4'):
                # Construct full file path
                src = os.path.join(dirpath, filename)
                des = os.path.join(dirpath, filename)
                
                # Process the video and save it
                convert_to_yolo_annotated_vid(model, src, des)

In [ ]:
from ultralytics import YOLO

# Load the YOLOv8 pose model
model = YOLO('yolov8l-pose.pt')

# Define the root directory where the train, val, and test folders are located
root_dir = '/kaggle/working/cricshot-yolo-ano'

# Process all videos in the directory tree
process_videos_in_directory(model, root_dir)

In [ ]:
# src = '/kaggle/working/cricshot-yolo-ano/test/lofted/lofted_0014.mp4'
# des = src
# convert_to_yolo_annotated_vid(model,src,des)

# src = '/kaggle/working/cricshot-yolo-ano/sweep/sweep_0012.mp4'
# convert_to_yolo_annotated_vid(model,src,src)

In [ ]:
import cv2
import matplotlib.pyplot as plt
from IPython.display import Video

# Path to the video file
video_path = '/kaggle/working/cricshot-yolo-ano/cover/cover_0101.mp4'

# Display the video in the notebook
Video(src, embed=True)


In [ ]:
import cv2
import matplotlib.pyplot as plt
from IPython.display import Video

# Path to the video file
video_path = '/kaggle/working/cricshot-yolo-ano/cover/cover_0101.mp4'

# Display the video in the notebook
Video(video_path, embed=True)


In [ ]:
import os
import shutil

def clear_directory_except(root_dir, folder_to_keep):
    for item in os.listdir(root_dir):
        item_path = os.path.join(root_dir, item)
        if item == folder_to_keep:
            continue
        if os.path.isdir(item_path):
            shutil.rmtree(item_path)
        else:
            os.remove(item_path)

# Example usage
root_directory = '/kaggle/working/'
folder_to_preserve = 'cricshot-yolo-ano'
clear_directory_except(root_directory, folder_to_preserve)
